In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
import os
from langsmith import traceable
import json
import asyncio
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
AZURE_OPENAI_API_KEY=os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT=os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_VERSION=os.environ["AZURE_OPENAI_API_VERSION"]

In [3]:
kernel = Kernel()

service = AzureChatCompletion(
    deployment_name="gpt-4o-mini",
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

kernel.add_service(service)

In [4]:
tech_prompt = "Answer from a system architecture perspective: {{$input}}"
request_settings_tech = AzureChatPromptExecutionSettings(temperature=0,max_tokens=100)
tech_function = kernel.add_function(plugin_name="parallel", function_name="tech_view", 
                                    prompt=tech_prompt, prompt_execution_settings=request_settings_tech)

In [5]:
biz_prompt = "Answer from a business impact perspective: {{$input}}"
request_settings_business = AzureChatPromptExecutionSettings(temperature=0,max_tokens=100)
biz_function = kernel.add_function(plugin_name="parallel", function_name="biz_view", 
                                   prompt=biz_prompt,prompt_execution_settings=request_settings_business)

In [6]:
risk_prompt = "Answer focusing on risks and limitations: {{$input}}"
request_settings_risk = AzureChatPromptExecutionSettings(temperature=0,max_tokens=100)
risk_function = kernel.add_function(plugin_name="parallel", function_name="risk_view", 
                                    prompt=risk_prompt,prompt_execution_settings=request_settings_risk)

In [7]:
from langsmith import traceable
import asyncio

@traceable(name="Tech Perspective")
async def run_tech(q):
    return await kernel.invoke(tech_function, input=q)

@traceable(name="Business Perspective")
async def run_biz(q):
    return await kernel.invoke(biz_function, input=q)

@traceable(name="Risk Perspective")
async def run_risk(q):
    return await kernel.invoke(risk_function, input=q)


In [8]:
@traceable(name="Parallel Agents Stage")
async def run_parallel_agents(q):

    tech_task = asyncio.create_task(run_tech(q))
    biz_task  = asyncio.create_task(run_biz(q))
    risk_task = asyncio.create_task(run_risk(q))

    tech_res, biz_res, risk_res = await asyncio.gather(
        tech_task, biz_task, risk_task
    )

    return {
        "tech": str(tech_res),
        "biz": str(biz_res),
        "risk": str(risk_res)
    }


In [15]:
merge_prompt = """
You are a synthesis agent.

Combine these perspectives into one coherent answer:

TECH:
{{$tech}}

BUSINESS:
{{$biz}}

RISK:
{{$risk}}
"""
request_settings_merger = AzureChatPromptExecutionSettings(temperature=0,max_tokens=500)
merge_function = kernel.add_function(plugin_name="parallel", 
                                     function_name="merge_agent", prompt=merge_prompt,
                                     prompt_execution_settings=request_settings_merger)

@traceable(name="Merge Step")
async def merge_answers(results):
    return await kernel.invoke(
        merge_function,
        tech=results["tech"],
        biz=results["biz"],
        risk=results["risk"]
    )

In [16]:
@traceable(name="Parallelization Agent")
async def parallel_agent(user_question):

    results = await run_parallel_agents(user_question)
    final_answer = await merge_answers(results)

    return final_answer

In [17]:
final_answer = await parallel_agent(
    "How does hybrid retrieval improve RAG systems?"
)

In [18]:
print(final_answer)

Hybrid retrieval is a transformative approach that significantly enhances Retrieval-Augmented Generation (RAG) systems by integrating various retrieval methods, thereby improving both the quality of information retrieved and the overall user experience. This synthesis of perspectives highlights the multifaceted benefits and considerations associated with hybrid retrieval.

### Advantages of Hybrid Retrieval in RAG Systems

1. **Combining Strengths of Different Retrieval Methods**: By leveraging both traditional information retrieval techniques, such as keyword-based searches (e.g., TF-IDF, BM25), and advanced semantic search methods, hybrid retrieval capitalizes on the strengths of each approach. This combination allows for more accurate and contextually relevant results, which is crucial for generating high-quality outputs.

2. **Improved Accuracy and Relevance**: The integration of diverse retrieval methods leads to enhanced accuracy in the information retrieved. This improvement not